In [ ]:
#dataset generation 
import numpy as np
import pandas as pd

# Set a seed for reproducibility
np.random.seed(0)

# Number of samples
n_samples = 100

# Generate a base feature
base_feature = np.random.normal(0, 1, n_samples)

# Generate highly correlated features by adding slight noise to the base feature
X = np.zeros((n_samples, 7))
for i in range(7):
    X[:, i] = base_feature + np.random.normal(0, 0.1, n_samples)

# Convert to DataFrame for easier handling and naming
feature_names = [f"feature_{i+1}" for i in range(7)]
df = pd.DataFrame(X, columns=feature_names)

# Generate a target variable with some dependence on the features
true_coefficients = np.array([3, -2, 1.5, -4, 2.5, -1, 0.5])  # Coefficients for the features
y = X.dot(true_coefficients) + np.random.normal(0, 0.5, n_samples)  # Add some noise to the target

# Add target variable to the DataFrame
df['target'] = y
print(df.head())

   feature_1  feature_2  feature_3  feature_4  feature_5  feature_6  \
0   1.952367   1.727134   1.633400   1.704187   1.802326   1.609009   
1   0.265381   0.376219   0.565970   0.288568   0.396733   0.441889   
2   0.851689   1.088704   0.966922   1.055404   1.088373   0.884301   
3   2.337833   2.306420   2.172875   2.276522   2.217472   2.264704   
4   1.750246   1.931571   1.934196   1.690704   1.832813   1.726962   

   feature_7    target  
0   1.619558  2.448355  
1   0.279103  0.820787  
2   0.899871 -0.135743  
3   2.350357  0.812428  
4   1.891040  1.797122  


In [10]:
#function for ridge regression
def ridge_regression(X,y,alpha=0.01,learning_rate=1e-15,epochs=1000):
    n,k=X.shape
    theta=np.zeros(k)

    cost_history=[]

    for epoch in range(epochs):
        y_pred=X.dot(theta)
        error=y_pred-y
        mse=(1/(2*n)*(np.sum(error**2)))

        regularization=(alpha/(2*n))*np.sum(theta[1:]**2)

        cost=mse+regularization
        cost_history.append(cost)

        gradient=(1/n)*X.T.dot(error) +(alpha/n)*np.r_[0,theta[1:]]

        theta=theta-learning_rate*gradient

        if epoch %100==0:
          print(f"Epoch {epoch}, Cost: {cost}")

        return theta,cost_history

In [14]:
from sklearn.metrics import r2_score
X = np.hstack([np.ones((n_samples, 1)), df[feature_names].values])#here we are adding 1's to the column so that we can get 1 for the intercept term
y = df['target'].values

learning_rates = [0.0001, 0.001, 0.01, 0.1, 1, 10]
regularization_params = [1e-15, 1e-10, 1e-5, 1e-3, 0, 1, 10, 20]

best_r2 = -np.inf  # Initialize with lowest possible R2
best_params = {'learning_rate': None, 'alpha': None, 'theta': None, 'cost': None}

# Grid Search
for lr in learning_rates:
    for alpha in regularization_params:
        theta, final_cost = ridge_regression(X, y, alpha, lr)
        
        # Predictions and R2 Score calculation
        y_pred = X.dot(theta)
        r2 = r2_score(y, y_pred)
        
        # Track best parameters
        if r2 > best_r2:
            best_r2 = r2
            best_params['learning_rate'] = lr
            best_params['alpha'] = alpha
            best_params['theta'] = theta
            best_params['cost'] = final_cost

print("Best Parameters:")
print("Learning Rate:", best_params['learning_rate'])
print("Regularization Parameter (alpha):", best_params['alpha'])
print("Minimum Cost:", best_params['cost'])
print("Maximum R^2 Score:", best_r2)

Epoch 0, Cost: 0.5073832781442862
Epoch 0, Cost: 0.5073832781442862
Epoch 0, Cost: 0.5073832781442862
Epoch 0, Cost: 0.5073832781442862
Epoch 0, Cost: 0.5073832781442862
Epoch 0, Cost: 0.5073832781442862
Epoch 0, Cost: 0.5073832781442862
Epoch 0, Cost: 0.5073832781442862
Epoch 0, Cost: 0.5073832781442862
Epoch 0, Cost: 0.5073832781442862
Epoch 0, Cost: 0.5073832781442862
Epoch 0, Cost: 0.5073832781442862
Epoch 0, Cost: 0.5073832781442862
Epoch 0, Cost: 0.5073832781442862
Epoch 0, Cost: 0.5073832781442862
Epoch 0, Cost: 0.5073832781442862
Epoch 0, Cost: 0.5073832781442862
Epoch 0, Cost: 0.5073832781442862
Epoch 0, Cost: 0.5073832781442862
Epoch 0, Cost: 0.5073832781442862
Epoch 0, Cost: 0.5073832781442862
Epoch 0, Cost: 0.5073832781442862
Epoch 0, Cost: 0.5073832781442862
Epoch 0, Cost: 0.5073832781442862
Epoch 0, Cost: 0.5073832781442862
Epoch 0, Cost: 0.5073832781442862
Epoch 0, Cost: 0.5073832781442862
Epoch 0, Cost: 0.5073832781442862
Epoch 0, Cost: 0.5073832781442862
Epoch 0, Cost:

In [15]:
df=pd.read_csv("Hitters (1).csv")

In [16]:
df.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N


In [19]:
df.isnull().sum()

AtBat         0
Hits          0
HmRun         0
Runs          0
RBI           0
Walks         0
Years         0
CAtBat        0
CHits         0
CHmRun        0
CRuns         0
CRBI          0
CWalks        0
League        0
Division      0
PutOuts       0
Assists       0
Errors        0
Salary       59
NewLeague     0
dtype: int64

In [20]:
# Drop rows with missing values
data = df.dropna()

# Convert categorical columns to numerical using one-hot encoding
data = pd.get_dummies(data, drop_first=True)

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Separate input and output features
X = data.drop('Salary', axis=1)
y = data['Salary']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [22]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso

# Initialize the models with specified parameters
linear_model = LinearRegression()
ridge_model = Ridge(alpha=0.5748)
lasso_model = Lasso(alpha=0.5748)

# Train each model
linear_model.fit(X_train, y_train)
ridge_model.fit(X_train, y_train)
lasso_model.fit(X_train, y_train)

C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.185e+04, tolerance: 4.367e+03
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=0.5748)

In [24]:
from sklearn.metrics import r2_score, mean_absolute_error

# Make predictions
y_pred_linear = linear_model.predict(X_test)
y_pred_ridge = ridge_model.predict(X_test)
y_pred_lasso = lasso_model.predict(X_test)

# Evaluate each model
r2_linear = r2_score(y_test, y_pred_linear)
r2_ridge = r2_score(y_test, y_pred_ridge)
r2_lasso = r2_score(y_test, y_pred_lasso)

mse_linear = mean_absolute_error(y_test, y_pred_linear)
mse_ridge = mean_absolute_error(y_test, y_pred_ridge)
mse_lasso = mean_absolute_error(y_test, y_pred_lasso)

# Print the evaluation results
print("Linear Regression: R^2 =", r2_linear, ", MSE =", mse_linear)
print("Ridge Regression: R^2 =", r2_ridge, ", MSE =", mse_ridge)
print("Lasso Regression: R^2 =", r2_lasso, ", MSE =", mse_lasso)


Linear Regression: R^2 = 0.2907451855798151 , MSE = 211.11596204561255
Ridge Regression: R^2 = 0.30003596988293757 , MSE = 204.66417420122414
Lasso Regression: R^2 = 0.29928590166965496 , MSE = 206.20588725930833


In [26]:

import pandas as pd

# Load the Boston housing dataset
df=pd.read_csv('Boston_Housing.csv')


In [27]:
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [28]:
y=df['MEDV']
X=df.drop('MEDV',axis=1)

In [29]:
from sklearn.model_selection import train_test_split

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
from sklearn.preprocessing import StandardScaler

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [31]:
from sklearn.linear_model import RidgeCV, LassoCV

# Define a range of alphas for cross-validation
alphas = [1e-15, 1e-10, 1e-5, 1e-3, 0.1, 1, 10, 20]

# Ridge regression with cross-validation
ridge_cv = RidgeCV(alphas=alphas, store_cv_values=True)
ridge_cv.fit(X_train, y_train)

# Lasso regression with cross-validation
lasso_cv = LassoCV(alphas=alphas, max_iter=10000, cv=5)
lasso_cv.fit(X_train, y_train)


C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2523.376596859873, tolerance: 2.8821090464396293
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1555.8220121814525, tolerance: 2.764448730650155
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:617: ConvergenceWarning: Obj

LassoCV(alphas=[1e-15, 1e-10, 1e-05, 0.001, 0.1, 1, 10, 20], cv=5,
        max_iter=10000)

In [32]:
from sklearn.metrics import r2_score, mean_squared_error

# Ridge model predictions and evaluation
ridge_predictions = ridge_cv.predict(X_test)
ridge_r2 = r2_score(y_test, ridge_predictions)
ridge_mse = mean_squared_error(y_test, ridge_predictions)

# Lasso model predictions and evaluation
lasso_predictions = lasso_cv.predict(X_test)
lasso_r2 = r2_score(y_test, lasso_predictions)
lasso_mse = mean_squared_error(y_test, lasso_predictions)

# Print results
print("Ridge Regression:")
print("Best alpha:", ridge_cv.alpha_)
print("R^2 score:", ridge_r2)
print("MSE:", ridge_mse)

print("\nLasso Regression:")
print("Best alpha:", lasso_cv.alpha_)
print("R^2 score:", lasso_r2)
print("MSE:", lasso_mse)


Ridge Regression:
Best alpha: 10.0
R^2 score: 0.6659677905050475
MSE: 24.495845619666415

Lasso Regression:
Best alpha: 1e-15
R^2 score: 0.6687594935356338
MSE: 24.29111947497339
